# MLops
Door Mark Olieman en Tommi Lander

Dit notebook bevat alles wat ook in de .py bestanden staat. Echter staat in de requirements voor het nakijken op brightspace dat het in dit formaat moet staan anders wordt het niet nagekeken. Dus bij deze dit formaat.

In [6]:
%pip install numpy pandas scikit-learn opencv-python cvzone ultralytics pyyaml moviepy
%pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118 
# deze import is om local te kunnen runnen op je gpu
# als je de local gpu import meeneemt kan deze cel 20+ minuten duren om te runnen

  Using cached moviepy-1.0.3-py3-none-any.whl
  Using cached decorator-4.4.2-py2.py3-none-any.whl.metadata (4.2 kB)
  Using cached proglog-0.1.10-py3-none-any.whl.metadata (639 bytes)
  Using cached imageio-2.34.1-py3-none-any.whl.metadata (4.9 kB)
  Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl.metadata (1.6 kB)
Using cached decorator-4.4.2-py2.py3-none-any.whl (9.2 kB)
Using cached imageio-2.34.1-py3-none-any.whl (313 kB)
Using cached imageio_ffmpeg-0.5.1-py3-none-win_amd64.whl (22.6 MB)
Using cached proglog-0.1.10-py3-none-any.whl (6.1 kB)
  Attempting uninstall: decorator
    Found existing installation: decorator 5.1.1
    Uninstalling decorator-5.1.1:
      Successfully uninstalled decorator-5.1.1
Note: you may need to restart the kernel to use updated packages.
Looking in indexes: https://download.pytorch.org/whl/cu118Note: you may need to restart the kernel to use updated packages.



# MLops

## TODO: inhoudsopgave

### 1. Imports

In [7]:
import random
import math
import shutil
import yaml # type: ignore
import os

import tkinter as tk
from tkinter import filedialog

import numpy as np # type: ignore
import pandas as pd # type: ignore

import moviepy.editor as mp # type: ignore
from ultralytics import YOLO # type: ignore
import cv2 # type: ignore
import cvzone # type: ignore



### 2. Labeler: labeler.py

### Uitleg van de code

Deze code bevat twee hoofdonderdelen: een functie om een video te resizen en een klasse `imagePainter` die frames uit een video haalt en de mogelijkheid biedt om rechthoeken op deze frames te tekenen en op te slaan.

#### Functie: `resize_video`

De `resize_video` functie neemt een pad naar een video als invoer, resize de video naar een resolutie van 1280x720, en slaat de resized video op in de map `new_data/resized_videos`. Hier zijn de belangrijkste stappen:

1. Laad de video met `VideoFileClip` uit de `moviepy` bibliotheek.
2. Bepaal het nieuwe pad voor de resized video.
3. Resize de video naar 1280x720 pixels.
4. Sla de resized video op met de codec 'libx264'.
5. Geef het pad naar de nieuwe video terug.

#### Class: `imagePainter`

De `imagePainter` klasse heeft als doel om frames uit een video te halen en rechthoeken op deze frames te tekenen. Hieronder volgt een uitleg van de belangrijkste methoden en attributen:

- **Constructor (`__init__`)**:
  - Initialiseer de klasse met het pad naar de video, het startframe, en een optionele titel.
  - Stel paden en namen in voor de video en de coördinaten van de rechthoeken.
  - Controleer of er al een CSV-bestand met coördinaten bestaat, zo niet, maak een nieuw DataFrame aan.
  - Roep de methode `draw_on_frames_from_video` aan om frames te verwerken.

- **Methode: `create_folders`**:
  - Maak mappen aan voor de data, frames en face-frames als deze nog niet bestaan.
  - Gebruik `check_and_create_folder` om te controleren of de map al bestaat en maak deze aan indien nodig.

- **Methode: `check_and_create_folder`**:
  - Probeer een map aan te maken en geef een melding als de map al bestaat.

- **Methode: `get_data_paths`**:
  - Bepaal de paden voor het opslaan van frames en face-frames op basis van het framenummer.

- **Methode: `draw_on_frames_from_video`**:
  - Verwerk de video frame voor frame vanaf het opgegeven startframe.
  - Laat de gebruiker rechthoeken tekenen op de frames.
  - Sla de frames en rechthoekcoördinaten op in CSV-bestanden.

- **Methode: `image_painter`**:
  - Voor elk frame, laat de gebruiker rechthoeken tekenen en sla de resultaten op.
  - Stel een muiscallback in om rechthoeken te kunnen tekenen (`draw_rectangle_with_drag`).

- **Methode: `draw_rectangle_with_drag`**:
  - Handelt muisgebeurtenissen af om rechthoeken te tekenen op de frames.
  - Slaat de coördinaten van de getekende rechthoek op in een DataFrame.

De cel hieronder doet alles om het train proces te kunnen uitvoeren, alles behalve het kiezen van de video. Dat wordt in de app uitgevoerd.


In [ ]:
def resize_video(path):
    clip = mp.VideoFileClip(path)
    path = os.path.basename(path)
    new_path = f'new_data/resized_videos/{path}'
    clip_resized = clip.resize((1280, 720))
    clip_resized.write_videofile(new_path, codec='libx264')
    return new_path
    
class imagePainter:
    def __init__(self, video_path, frame_start, title='ImagePainter!'):
        self.running = True
        
        self.title = title
        self.frame_start = frame_start
        
        self.video_name = os.path.basename(video_path).rstrip('.mp4')
        self.video_path = video_path.rstrip(self.video_name)
        self.coords_path = f'new_data/coords.csv'
        
        if 'coords.csv' not in os.listdir('new_data/'):
            self.rectangle_coords = pd.DataFrame({'image': [], 'x1': [], 'y1': [], 'x2': [], 'y2': []})
            self.rectangle_coords.to_csv(self.coords_path)
        else:
            self.rectangle_coords = pd.read_csv(self.coords_path)
        
        self.draw_on_frames_from_video(video_path)
        
    def create_folders(self):
        data_folder = 'data'
        frame_folder = f'{data_folder}/{self.video_name}_frames'
        face_folder =  f'{frame_folder}_face_boxes'

        self.check_and_create_folder(data_folder)
        self.check_and_create_folder(frame_folder)
        self.check_and_create_folder(face_folder)
        
        return (frame_folder, face_folder, data_folder)
    
    def check_and_create_folder(self, folder):
        try:
            os.mkdir(folder)
            print(f'Creating {folder}')
        except FileExistsError:
            print(f'{folder} folder already exists! skipping creation...')
            
    def get_data_paths(self, count):
        frame_path = f'new_data/frames/{self.video_name}_frame_{count}.jpg'
        face_path = f'new_data/face_frames/{self.video_name}_face_frame_{count}.jpg'
        
        return (frame_path, face_path)
             
    def draw_on_frames_from_video(self, video_path):
        self.show_previous_frame = False
        vid = cv2.VideoCapture(video_path)
        vid.set(cv2.CAP_PROP_POS_FRAMES, self.frame_start)
        frame_count = int(vid.get(cv2.CAP_PROP_FRAME_COUNT))
        count = self.frame_start
        
        while self.running and vid.isOpened():
            self.title = f'ImagePainter - {self.video_name} - Image{count}/{frame_count}'  
            if self.show_previous_frame:
                count -= 1
                vid.set(cv2.CAP_PROP_POS_FRAMES, current_frame-1)
                self.running, self.img = vid.read()
                self.show_previous_frame = False
            else:
                self.running, self.img = vid.read()
                
                
            next_frame = vid.get(cv2.CAP_PROP_POS_FRAMES)
            current_frame = next_frame - 1
            
            try:
                self.original_img = self.img.copy()
            except AttributeError:
                cv2.destroyAllWindows() 
                self.rectangle_coords.to_csv(self.coords_path, index=False)
                break
            self.drawing_img = self.img.copy()

            self.image_painter(count)
            
            count += 1
            
        self.rectangle_coords.to_csv(self.coords_path, index=False)
        
    def image_painter(self, count):
        self.image_name = f'{self.video_name}_frame_{count}.jpg'
        frame_path, face_path = self.get_data_paths(count)
        
        self.ix = -1
        self.iy = -1
        self.drawing = False
        
        while self.running:
            cv2.namedWindow(winname = self.title) 
            cv2.setMouseCallback(self.title,  self.draw_rectangle_with_drag) 
            cv2.imshow(self.title, self.img)
                
            key = cv2.waitKey(0)
            if key == 27:
                self.running = False
                cv2.destroyAllWindows() 
                self.rectangle_coords.to_csv(self.coords_path, index=False)
            else:
                cv2.imwrite(frame_path, self.original_img)
                cv2.imwrite(face_path, self.drawing_img)
                cv2.destroyAllWindows()  
                break
    
    def draw_rectangle_with_drag(self, event, x, y, flags, param):
        if event == cv2.EVENT_LBUTTONDOWN: 
            self.drawing = True
            self.ix = x 
            self.iy = y
            
        elif event == cv2.EVENT_MOUSEMOVE: 
            if self.drawing == True: 
                self.drawing_img = self.img.copy()
                cv2.rectangle(self.drawing_img, 
                              pt1=(self.ix, self.iy), pt2=(x, y), 
                              color=(0, 0, 255), 
                              thickness=1, 
                              lineType=cv2.LINE_AA)
                cv2.imshow(self.title, self.drawing_img)
        
        elif event == cv2.EVENT_LBUTTONUP: 
            self.img = self.drawing_img
            coords = pd.DataFrame({'image': [self.image_name], 'x1': [self.ix], 'y1': [self.iy], 'x2': [x], 'y2': [y]})
            self.rectangle_coords = pd.concat([self.rectangle_coords, coords], axis=0)
            self.drawing = False
            

## 3. Data verwerking: data.py

## Uitleg van de code

Deze code bevat verschillende functies en methoden om een dataset voor te bereiden voor een YOLOv1 modeltraining. Hieronder volgt een uitleg van de verschillende onderdelen van de code.

### Functie: `csv_to_labels`

Deze functie verwerkt een CSV-bestand met annotaties van bounding boxes om de data geschikt te maken voor invoer in een YOLOv8n model.

#### Stappen:

1. **Lees de CSV**:
   - Laad de CSV met de annotaties in een DataFrame.
   
2. **Bereken de middelpuntcoördinaten en afmetingen**:
   - Bereken de middenpunten (`x`, `y`) en de breedte en hoogte (`w`, `h`) van de bounding boxes.
   
3. **Pas coördinaten aan als ze buiten het beeld vallen**:
   - Corrigeer waarden die buiten de beeldresolutie van 1280x720 vallen.
   
4. **Normaliseer de coördinaten**:
   - Deel de coördinaten door de breedte en hoogte van het beeld om ze te normaliseren.
   
5. **Schrijf de aangepaste annotaties naar bestanden**:
   - Voor elke rij in de DataFrame, schrijf de genormaliseerde coördinaten naar een tekstbestand in de map `new_data/labels`.

### Functie: `create_training_run`

Deze functie bereidt een nieuwe trainingssessie voor door de benodigde mappen en bestanden aan te maken en de gegevens op te splitsen in trainings- en validatiesets.

#### Stappen:

1. **Verwijder bestaande face-frames en resized video’s**:
   - Verwijder alle bestanden in de mappen `new_data/face_frames` en `new_data/resized_videos`.
   
2. **Verwijder het bestaande coördinaten CSV-bestand**:
   - Probeer `new_data/coords.csv` te verwijderen.
   
3. **Maak een nieuwe trainingsmap aan**:
   - Bepaal het nummer voor de nieuwe trainingsrun en maak de bijbehorende mappen aan.
   
4. **Splits de data in training en validatie**:
   - Verdeel de beelden willekeurig in een trainings- en validatieset (80% training, 20% validatie).
   
5. **Verplaats de bestanden naar de juiste mappen**:
   - Verplaats de afbeeldingen en hun labels naar de juiste trainings- en validatiemappen.
   
6. **Maak een YAML-bestand aan**:
   - Roep `create_yaml` aan om een YAML-configuratiebestand te maken voor de trainingsrun.

### Functie: `get_highest_training_run`

Deze functie bepaalt het hoogste nummer van de trainingsrun die al bestaat.

#### Stappen:

1. **Lees de bestaande trainingsruns**:
   - Lees alle mappen in `Training_runs` en haal de hoogste numerieke suffix op.
   
2. **Geef het hoogste trainingsrun-nummer terug**:
   - Geef de naam van de trainingsrun met het hoogste nummer terug.

### Functie: `create_yaml`

Deze functie maakt een YAML-configuratiebestand aan voor de meest recente trainingsrun.

#### Stappen:

1. **Bepaal de hoogste trainingsrun**:
   - Roep `get_highest_training_run` aan om de naam van de meest recente trainingsrun te krijgen.
   
2. **Maak de YAML-inhoud aan**:
   - Maak de inhoud van het YAML-bestand aan met de paden naar de trainings- en validatiemappen en de naam van de klasse (`face`).
   
3. **Schrijf het YAML-bestand**:
   - Schrijf de YAML-inhoud naar een bestand in de map van de hoogste trainingsrun.


In [ ]:
def csv_to_labels():
  """
  Preprocesses a CSV file containing bounding box annotations to prepare
  the data for YOLOv1 model input with a 16x9 grid size.

  Args:
      csv_path (str): Path to the CSV file containing bounding box annotations.
      image_size (tuple): Size of the original images (height, width).

  Returns:
      tuple: Tuple of numpy arrays (X, Y). X represents the grids and Y represents the bounding boxes.
  """
  image_w, image_h = (1280, 720)
  df = pd.read_csv('new_data/coords.csv')
  
  df['x'] = (df['x1']+df['x2']) /2
  df['y'] = (df['y1']+df['y2']) /2
  df['w'] = df['x2'] - df['x1']
  df['h'] = df['y2'] - df['y1']
  
  df = df.drop(columns=['x1', 'y1', 'x2', 'y2'])
  
  df['y'] = df['y'].apply(lambda x: 1279 if x >= 1280 else x)
  df['x'] = df['x'].apply(lambda x: 719 if x >= 720 else x)

  # Normalize bounding box coordinates
  df['x'] = df['x'] / image_w
  df['y'] = df['y'] / image_h
  df['w'] = df['w'] / image_w
  df['h'] = df['h'] / image_h
    
  for _, row in df.iterrows():
    path = 'new_data/labels/' + row['image']
    path = path.rstrip('.jpg')
    with open(path + '.txt', 'a') as file:
      x = row['x']
      y = row['y']
      w = row['w']
      h = row['h']
      file.write(f'0 {x} {y} {w} {h}')
      file.write('\n')

def create_training_run():
  face_frames_folder = 'new_data/face_frames/'
  resized_videos_folder = 'new_data/resized_videos/'
  
  for face_frame in os.listdir(face_frames_folder):
    file_path = os.path.join(face_frames_folder, face_frame)
    os.unlink(file_path)
    
  for video in os.listdir(resized_videos_folder):
    file_path = os.path.join(resized_videos_folder, video)
    os.unlink(file_path)
  
  try:
    os.unlink('new_data/coords.csv')
  except:
    pass
    
  training_folder = 'Training_runs'
  
  image_folder = 'new_data/frames'
  label_folder = 'new_data/labels'
  
  counter = 1
  new_training_path = f"{training_folder}/training_run_{counter}"
  
  
  
  while os.path.exists(new_training_path):
      counter += 1
      new_training_path = f"{training_folder}/training_run_{counter}"
  os.makedirs(new_training_path)
  
  train_image_folder = f'{new_training_path}/train/images'
  val_image_folder = f'{new_training_path}/val/images'
  train_label_folder = f'{new_training_path}/train/labels'
  val_label_folder = f'{new_training_path}/val/labels'
  
  os.makedirs(train_image_folder)
  os.makedirs(val_image_folder)
  os.makedirs(train_label_folder)
  os.makedirs(val_label_folder)
  
  images = [f for f in os.listdir(image_folder)]
  
  # Shuffle images for random splitting
  random.shuffle(images)
  
  # Determine the split index
  split_index = int(len(images) * (1 - 0.2))

  # Split into training and validation sets
  train_images = images[:split_index]
  val_images = images[split_index:]

  # Move files to their respective directories
  for img in train_images:
      shutil.move(os.path.join(image_folder, img), os.path.join(train_image_folder, img))
      shutil.move(os.path.join(label_folder, os.path.splitext(img)[0] + '.txt'), os.path.join(train_label_folder, os.path.splitext(img)[0] + '.txt'))
  
  for img in val_images:
      shutil.move(os.path.join(image_folder, img), os.path.join(val_image_folder, img))
      shutil.move(os.path.join(label_folder, os.path.splitext(img)[0] + '.txt'), os.path.join(val_label_folder, os.path.splitext(img)[0] + '.txt'))
    
  create_yaml()
  
def get_highest_training_run():
    training_run_folders = [f for f in os.listdir('Training_runs/')]
    training_run_numbers = [int(f.split('_')[-1]) for f in training_run_folders if f.split('_')[-1].isnumeric()]
    try:
        highest_number = max(training_run_numbers)
    except:
        highest_number = 0
    return f"training_run_{highest_number}"

def create_yaml():
  highest_training_run = get_highest_training_run()
  yaml_content = {
      'path': 'training_runs',
      'train': f"{highest_training_run}/train",
      'val': f"{highest_training_run}/val",
      'names': {
          0: 'face'
      }
  }
  with open(os.path.join(f'Training_runs/{highest_training_run}', f'{highest_training_run}.yaml'), 'w') as yaml_file:
      yaml.dump(yaml_content, yaml_file, default_flow_style=False)



## 4. Training: train.py

## Uitleg van de code

De functie `train_model` traint een YOLO-model (You Only Look Once) op basis van de meest recente trainingsrun. Hieronder volgt een gedetailleerde uitleg van de verschillende stappen in de functie.

### Functie: `train_model`

#### Stappen:

1. **Schakel Weights & Biases uit**:
   - Zet de `WANDB_DISABLED` omgevingsvariabele op 'true' om te voorkomen dat Weights & Biases wordt gebruikt tijdens de training.
   
2. **Bepaal de meest recente trainingsrun**:
   - Roep de functie `get_highest_training_run` aan om de naam van de meest recente trainingsrun te krijgen.
   
3. **Bepaal het pad naar het YAML-configuratiebestand**:
   - Stel het pad in naar het YAML-bestand dat is aangemaakt voor de meest recente trainingsrun.
   
4. **Laad het YOLO-model**:
   - Laad een vooraf getraind YOLO-model met de beste gewichten (`best.pt`) uit een de laatste training run.
   
5. **Train het model**:
   - Probeer het model te trainen met de configuratie uit het YAML-bestand, voor 50 epochs, met enkele klasse (`single_cls=True`), in rechthoekige modus (`rect=True`), zonder plots (`plots=False`), en met een afbeeldingsgrootte van 720x1280 pixels.
   - Specificeer het project (`project='Training_runs'`) en de naam van de trainingssessie (`name=f'{highest_number_training_run}_training'`).
   - Probeer eerst de GPU (`device=0`) te gebruiken. Als dat mislukt, val dan terug op de CPU (`device='cpu'`).

### Overzicht

Deze functie zorgt ervoor dat het model wordt getraind met behulp van de gegevens en configuratie van de meest recente trainingsrun. Het gebruikt de YOLO-bibliotheek om het model te laden en te trainen, en behandelt eventuele fouten door automatisch over te schakelen van GPU naar CPU als dat nodig is.


In [ ]:
def train_model():
    os.environ['WANDB_DISABLED'] = 'true'
    highest_number_training_run = get_highest_training_run()
    
    yaml_path = f'Training_runs/{highest_number_training_run}/{highest_number_training_run}.yaml'
    # Load and train the YOLO model
    model = YOLO('Data/training/weights/best.pt')
    
    # Train the model using the provided YAML configuration
    try:
        model.train(data=yaml_path, epochs=50, single_cls=True, rect=True, plots=False, imgsz=[720, 1280], project=f'Training_runs', name=f'{highest_number_training_run}_training', device=0)
    except:
        model.train(data=yaml_path, epochs=50, single_cls=True, rect=True, plots=False, imgsz=[720, 1280], project=f'Training_runs', name=f'{highest_number_training_run}_training', device='cpu')

## 5. Detection: detection.py

## Uitleg van de code

De functie `show_detection` gebruikt een webcam om realtime objectdetectie uit te voeren met een getraind YOLO-model. Hieronder volgt de gedetailleerde uitleg van de verschillende stappen in de functie.

### Functie: `show_detection`

#### Stappen:

1. **Start de webcam**:
   - Gebruik `cv2.VideoCapture(0)` om de webcam te starten.
   - Stel de resolutie van de webcam in op 1280x720 pixels met `cap.set(3, 1280)` en `cap.set(4, 720)`.
   
2. **Bepaal de meest recente trainingsrun**:
   - Roep de functie `get_highest_training_run` aan om de naam van de meest recente trainingsrun te krijgen.

3. **Laad het YOLO-model**:
   - Als er geen trainingsruns zijn, laad dan de vooraf door ons getrainde model (`'Data/training/weights/best.pt'`)
   - Als er wel trainingsruns zijn, laad dan het model uit de meest recente trainingsrun (`{highest_training_run}_training/weights/best.pt`).

4. **Definieer klasse namen**:
   - Definieer een lijst met class names voor het model om te weten wat er allemaal gedetecteerd moet worden (`classNames`), in dit geval alleen "face".

5. **Lees frames van de webcam en voer detectie uit**:
   - Lees continu frames van de webcam met `cap.read()`.
   - Gebruik het YOLO-model om objecten in het frame te detecteren (`model(img, stream=True)`).
   - Voor elke detectie, teken een rechthoek om de gedetecteerde objecten met `cvzone.cornerRect`.
   - Voeg de klassenamen en vertrouwen (confidence) toe aan het beeld met `cvzone.putTextRect`.

6. **Toon het beeld**:
   - Toon het beeld met de getekende rechthoeken en tekst in een venster met `cv2.imshow("Image", img)`.

7. **Beëindig de detectie**:
   - Als de 'q'-toets wordt ingedrukt, stop dan de webcam en sluit alle vensters met `cap.release()` en `cv2.destroyAllWindows()`.

### Overzicht

Deze functie zorgt ervoor dat de webcam continu frames leest en deze frames worden verwerkt door een YOLO-model om objecten (gezichten) in realtime te detecteren. Gedetecteerde objecten worden gemarkeerd met rechthoeken en vertrouwen wordt weergegeven op het beeld. De detectie stopt wanneer de gebruiker op de 'q'-toets drukt.


In [ ]:
def show_detection():
    cap = cv2.VideoCapture(0)
    cap.set(3, 1280)
    cap.set(4, 720)
    
    highest_training_run = get_highest_training_run()
    
    if not os.listdir('Training_runs'):
        model = YOLO('Data/training/weights/best.pt')
    else:
        model = YOLO(f'Training_runs/{highest_training_run}_training/weights/best.pt')


    classNames = ["face"]

    while True:
        success, img = cap.read()
        results = model(img, stream=True)
        for r in results:
            boxes = r.boxes
            for box in boxes:
                x1, y1, x2, y2 = box.xyxy[0]
                x1, y1, x2, y2 = int(x1), int(y1), int(x2), int(y2)
                w, h = x2-x1, y2-y1
                cvzone.cornerRect(img, (x1, y1, w, h))

                conf = math.ceil((box.conf[0]*100))/100

                cls = box.cls[0]
                name = classNames[int(cls)]

                cvzone.putTextRect(
                    img, f'{name} 'f'{conf}', (max(0, x1), max(35, y1)), scale=2, thickness=2, 
                    colorT=(255,255,255), colorR=(54,250,74))

        cv2.imshow("Image", img)
        if cv2.waitKey(1) == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break

## 6. The App: app.py

## Uitleg van de code

Deze code maakt een eenvoudige GUI (Graphical User Interface) voor objectdetectie, met behulp van Tkinter. De GUI bevat drie knoppen die verschillende acties uitvoeren: het labelen van een nieuwe video, het hertrainen van het model en het detecteren van gezichten vanuit een livestream. Hieronder volgt een gedetailleerde uitleg van de verschillende onderdelen van de code.

### Functie: `button1_action`

Deze functie wordt uitgevoerd wanneer op de knop "Label new video" wordt gedrukt.

#### Stappen:

1. **Vraag de gebruiker om een videobestand te selecteren**:
   - Gebruik `filedialog.askopenfilename` om een bestandsdialoog te openen waarin de gebruiker een MP4-bestand kan selecteren.
   
2. **Pas de grootte van de video aan**:
   - Roep `resize_video(file_path)` aan om de geselecteerde video te verkleinen naar een resolutie van 1280x720 pixels.
   
3. **Start de `imagePainter`**:
   - Gebruik `imagePainter` om het labelen van de frames van de nieuwe video te starten vanaf frame 0.
   
4. **Converteer de gelabelde coördinaten naar YOLO-formaat**:
   - Roep `csv_to_labels()` aan om de coördinaten van de gelabelde gezichten om te zetten naar een formaat dat geschikt is voor YOLO.

### Functie: `button2_action`

Deze functie wordt uitgevoerd wanneer op de knop "Retrain model" wordt gedrukt.

#### Stappen:

1. **Maak een nieuwe trainingsrun aan**:
   - Roep `create_training_run()` aan om een nieuwe trainingsrun aan te maken en de benodigde mappen en bestanden in te stellen.
   
2. **Train het model opnieuw**:
   - Roep `train_model()` aan om het YOLO-model opnieuw te trainen met de nieuwste gegevens.

### Functie: `button3_action`

Deze functie wordt uitgevoerd wanneer op de knop "Detect faces from livestream" wordt gedrukt.

#### Stappen:

1. **Start de detectie van gezichten vanuit een livestream**:
   - Roep `show_detection()` aan om de webcam te starten en gezichten in realtime te detecteren en weer te geven.

### Hoofdvenster van de GUI

1. **Maak het hoofdvenster aan**:
   - Gebruik `tk.Tk()` om een nieuw hoofdvenster te maken en stel de titel in op "Object detection".
   - Stel de grootte van het venster in op 400x60 pixels.

2. **Maak knoppen aan**:
   - Maak drie knoppen aan: "Label new video", "Retrain model", en "Detect faces from livestream".
   - Koppel elke knop aan de bijbehorende functie (`button1_action`, `button2_action`, `button3_action`).
   - Plaats de knoppen in een gridlayout met `grid(column, row, pady, padx)`.

3. **Start de hoofdloop van de applicatie**:
   - Gebruik `root.mainloop()` om de GUI te starten en de applicatie in een continue lus te houden totdat deze wordt afgesloten door de gebruiker.

### Overzicht

Deze code maakt een eenvoudige GUI voor het uitvoeren van verschillende taken gerelateerd aan objectdetectie. Gebruikers kunnen een nieuwe video labelen, het model hertrainen, of gezichten detecteren vanuit een livestream door op de bijbehorende knoppen te drukken.


### App beschrijving:
We hebben ervoor gekozen om de layout van de app zo simpel mogelijk te houden, daarom hebben wij drie knoppen:

- "Label new video"
- "Retrain model"
- "Detect faces from livestream"

Hier leggen wij uit wat elke knop doet:

"label new video": Als eerst kies je jouw video uit die je wilt labelen, deze wordt vervolgens geresized en kun je beginnen met labelen door boxes te tekenen. Als je naar het volgende frame wilt gaan klik je op 'q' en als je klaar bent klik je op 'escape'. Nadat je klaar bent worden de images en labels opgeslagen in het mapje "new_data", hier kan je ook controleren of alle labeling goed is gegaan aangezien de afbeeldingen met bounding boxes ook opgeslagen worden. Deze map is tijdelijk en zodra de "Retrain model" knop gedrukt wordt, worden de images en labels verplaatst en de images met bounding boxes worden verwijderd. Dit wordt gedaan zodat je het model telkens opnieuw kan trainen met nieuwe data.

"Retrain model": Als je op retrain drukt wordt jouw gelabelde data gebruikt uit de map "new_data" en verplaatst naar "Training_runs". Daar komt een mapje genaamd training_run_{nummer} in waar de images en labels in yolo formaat worden neergezet, zodat dit gebruikt kan worden voor het trainen. Ook wordt er een .yaml bestand gemaakt die het model verteld waar de data staat om zo opnieuw te kunnen trainen. Dan wordt uiteindelijk het model getrained en komt er een mapje genaamd training_run_{nummer}_training met daarin de resultaten van de getrainde epochs, de weigths van de laatste epoch en de beste epoch en nog een args.yaml met de argumenten die het model heeft gebruikt tijdens het trainen.

"Detect faces from livestream": Hiervoor wordt het model gerund, als je nog niks hebt getrained dan word het door ons getrainde model gebruikt. Als je wel zelf hebt getrained wordt de laatste training run gebruikt. De livestream start en laat de boxes met labels zien waar de gezichten worden gedetecteerd.

In [ ]:
def button1_action():
    file_path = filedialog.askopenfilename(
        filetypes=[("Image Files", "*.mp4")]
    )
    new_path = resize_video(file_path)
    imagePainter(new_path, 0)
    csv_to_labels()
    
def button2_action():
    create_training_run()
    train_model()

def button3_action():
    show_detection()

# Create the main window
root = tk.Tk()
root.title("Object detection")
root.geometry('400x60')


# Create buttons
button1 = tk.Button(root, text="Label new video", command=button1_action)
button1.grid(column=0, row=1, pady=15, padx=10)

button2 = tk.Button(root, text="Retrain model", command=button2_action)
button2.grid(column=1, row=1, pady=15, padx=10)

button3 = tk.Button(root, text="Detect faces from livestream", command=button3_action)
button3.grid(column=2, row=1, pady=15, padx=10)

# Run the application
root.mainloop()


## 7. Duidelijk overzicht en omschrijving van het product.
Onze AI-app maakt gebruik van een fine-tuned YOLOv8 model wat een state-of-the-art objectdetectie-model is om zo live objectdetectie(faces) te voorspellen op het apparaat waar het gerunt word. In deze app kun je zelf labelen en hertrainen van je model! Deze app zorgt ervoor dat iedereen in staat is om simpel en goed gezichten te detecteren!

Onze app maakt gebruik van een voorgetrained deep learning model waardoor beginners en experts:
Zelf data kunnen labelen, voor bijvoorbeeld eigen datasets of als je het model simpelweg nog beter wilt maken. Dit zorgt ervoor dat je alle controle hebt over je traingegevens en zo bepaal ook jij je nauwkeurigheid van het model. De app kan live gezichten detecteren door je eigen camera direct op je apparaat. Dit is erg handig als je snel en goede resultaten nodig hebt. 

Onze app is dus erg flexibel en geeft de gebruiker volledige controle van het labelen, hertrainen en het uiteindelijke voorspellen!
Zo is de app te gebruiken voor real-time detectie in cameras, ook bij autonome drones of beveiliging.

## 8. Lijst van de stakeholders
De app is bedoeld als oplossing voor stationaire camerabewaking daardoor zijn de stakeholders als volgt:
- de gebruikers van de app
- de ontwikkelaars van de app(wij)
- iedereen die mogelijk in de gebieden komt waar de app gebruikt wordt voor camera bewaking

## 9. Datavereisten gedetailleerd beschreven

De datavereisten van de app zijn als volgt:

- De data moet het formaat 720x1280 zijn, dit is het formaat waarop het model wordt getraind, echter kan de app zelf videos naar dit formaat resizen, maar dit kan wel zorgen voor distorties.
- Er moeten gezichten voorkomen in de data
- De data moet lijken op de omstandigheden waar je wilt gaan bewaken. 
- De data moet gevarieerd zijn met belichting, afstand van de gezichten, hoeveelheid gezichten en soort gezichten

Deze vereisten zijn belangrijk om te zorgen dat de training van het model daadwerkelijk verbetering oplevert. Als je namelijk een hoop videos gebruikt die op elkaar lijken om te labelen dan kan het zijn dat het model andere situaties niet goed kan gebruiken voor het detecteren van gezichten. Maar omdat het ook lastig is om overal aan te denken en goede data te verzamelen is het enorm makkelijk gemaakt om het model opnieuw te trainen, het model neemt dan de oude training runs mee, maar wordt verbeterd met de nieuwe data.

## 10. Modelvereisten gedetailleerd beschreven

Voor het model zelf zijn de eisen als volgt:

- de output moet betrouwbaar zijn, dus weinig foute positieven, dit omdat het de waarde van een daadwerkelijke predictie omlaag haalt. als een model iets mist kan je altijd zorgen dat de livestream opgeslagen wordt en deze terugkijken. Omdat mensen erg gevoelig zijn voor gezichtsherkenning is dit een makkelijke taak als er detecties missen, echter wordt het lastiger als er overal boxes worden getekent waar geen gezichten te zien zijn.
- de snelheid van het model moet onder de 500ms blijven, dit klinkt lang maar bewaking gaat voor lange periodes door en een lage framerate is niet erg aangezien mensen vaak langer dan een halve seconde in frame blijven. echter moet het niet langer duren dan 500ms ander is de kans te groot dat er toch detecties gemist worden.
- het model moet in kleur werken anders worden slecht belichtte omstandigheden te moeilijk om te verwerken
- het model moet kunnen runnen met minimale hardware, een cpu van een gemiddelde laptop zou hiervoor genoeg moeten zijn. Hierdoor wordt de app laagdrempelig en makkelijker in gebruik

Deze vereisten moeten ervoor waken dat het model goed blijft werken in diverse omstandigheden. Ook zorgen deze vereisten voor een model dat betrouwbaar is en blijft voldoen aan de verwachtingen van de gebruiker. Deze vereisten hebben wij toegepast op het voorgetrainde model, maar kunnen flink verbeterd worden als de gebruiker daarvoor kiest en het model hertraint

## 11. Monitoring- en hertrainingsstrategieën gedetailleerd beschreven

Monitoringsstrategie: Als gebruikers merken dat het model niet goed werkt of je wilt nieuwe data toevoegen. Je kunt de prestaties van het model bekijken en als daar uitkomt dat het model niet goed presteert bieden wij met onze app een handige hertrainingsoptie waarbij je simpel een eigen video in kunt laden en opnieuw kan hertrainen.

Hertrainingsstrategie:
Onze app zorgt voor een handige hertrianingsfunctie, waarmee consumenten erg makkelijk nieuwe data kunnen labelen. Met deze nieuwe data kun je uiteindelijk ook het model trainen zonder dat je uit de app gaat en de code in moet duiken. De "label new video" geeft de optie om eigen video's de labelen met locaties van gezichten. Deze knop maakt gebruik van een fijne interface voor consumenten om zo simpel eigen data te labelen met bounding boxes. Nadat je de data hebt gelabeld is er een "retrain model" knop die het model opnieuw laat trainen met de nieuwe data hier hoef je verder niks voor te doen!

Door deze hetrainingsstrategie wordt het model constant verbeterd aan nieuwe data waardoor het erg nauwkeurig blijft!

## 12. Uitleg datapipeline

De data pipeline werkt als volgt:
- de gebruiker klikt de knop "label new video"
- de video wordt geresized naar 720x1280 en opgeslagen in "new_data/resized_videos"
- de eerste frame van de video wordt getoond en de gebruiker kan doormiddel van de muis boxes tekenen
- als alle gezichten gelabeld zijn drukt de gebruiker op 'q' en wordt de volgende frame getoond, dit gaat zolang door als de gebruiker wilt of als er frames in de video zijn
- als de gebruiker klaar is drukt hij op 'escape', dit sluit de labeler af en slaat alles op
- in de achtergrond worden de frames, de frames met de getekende boxes, alle labels (.txt bestanden) en een csv met alle labels opgeslagen in verschillende mappen in "new_data"
- zodra de gebruiker genoeg videos heeft gelabeld en tevreden is drukt hij op "retrain model" en worden de frames(zonder getekende boxes) en labels (txt bestanden) verplaatst naar Training_runs folder. hier wordt het in een mapje gezet genaamd "Training_run_(nummer)" met als nummer de hoeveelste keer dat er getraint wordt. De images en labels worden in een formaat gezet dat bruikbaar is voor Yolo, dit houdt in een train en val folder met daarin de 2 folders, images en labels. in de train staan alle training data en in de val staat alle validation data. Ook wordt er een .yaml bestand aangemaakt dat ervoor zorgt dat het model kan zien waar deze folder zich bevinden en welke labels er getraind gaan worden.

De data pipeline is ingericht op een manier dat er zoveel als nodig gelabeld kan worden voordat de daadwerkelijke stap naar trainen wordt gezet. Dit is handig als er nog data verzameld moet worden, maar het labelen wel al gedeeltelijk gedaan kan worden. Ook slaan we de frames met getekende boxes op om te zien of het labelings-proces goed is uitgevoerd. verder hebben we een csv bestand met alle labels, dit maakt het makkelijk om als dat nodig is de labels makkelijk te exporteren als dat nodig is.

Dit opslaan vinden wij belangrijk voor de monitoring van het proces, hierdoor kan je goed in de gaten houden wat er gebeurd. echter vinden wij ook dat het overbodig is om alles oneindig te behouden, daarom worden de onnodige data verwijderd zodra er daadwerkelijk getraind wordt. 

## 13. Uitleg ML-pipeline

Ons trainingsproces bevindt zich in het bestand train.py.
In dit bestand wordt het model met de nieuwste data (uit data.py) getraind. Het gebruikt de beste gewichten die eerder zijn opgeslagen en past de nieuwe data toe om zo het model nog beter te maken. 
Het werkt als volgt: 
Het model dat als laatsts getraind is wordt geselecteerd en kijkt naar de laatst gegenereerde .yaml file, hier ziet hij waar hij de images en labels kan vinden. Met de images en labels gaat het model aan de slag en wordt er getraind op 50 epochs. Het model slaat vervolgens de laatste epoch en de beste epoch op in het mapje van de training run. Ook worden de arguments die gebruikt zijn bij het trainen opgeslagen om mogelijke problemen te kunnen onderzoeken. En als laatst wordt er een csv gegenereerd met de resultaten van de verschillende epochs.

## 14. Uitleg Deployment

Zoals eerder vermeld wordt de deployment gedaan door een app, deze app is makkelijk in gebruik en heeft alles wat je nodig hebt om aan de slag te gaan met gezichten herkennen. Door alle stappen van het labelen, trainen en detecteren in de app te verwerken kunnen wij een oplossing bieden voor snel veranderende data of doeleindes die specifieke data nodig heeft. En dit zonder enorm veel verschillende applicaties, programmas, of programmeurs te gebruiken. alles in 1 simpele oplossing. De app is makkelijk te openen door app.py te runnen, hierdoor is het simpel in gebruik. 

De app zelf heeft duidelijk gelabelde knoppen die precies doen wat je ervan verwacht namelijk:
"label new video", om een nieuwe video te labelen.
"retrain model", om het model opnieuw te trainen.
"Detect faces from livestream", om een livestream te openen met je webcam die vervolgens het laatste model gebruikt om gezichten te kunnen herkennen

De app is simpel en zorgt dus voor een laagdrempelige instap en een makkelijke ervaring zonder afleidingen

### Referentielijst
Computer vision zone. (2023, May 2). Train Yolov8 object detection on a custom dataset | Computer vision tutorial. YouTube. https://www.youtube.com/watch?v=m9fH9OWn8YM&t=3070s

PyTorch. (n.d.). Get started locally. https://pytorch.org/get-started/locally/

Ultralytics. (n.d.). Multi-GPU training. https://docs.ultralytics.com/nl/modes/train/#multi-gpu-training

Dalvi, T. (2019, November 14). Object detection with YOLO and OpenCV: A practical guide. Medium. https://medium.com/@tejasdalvi927/object-detection-with-yolo-and-opencv-a-practical-guide-cf7773481d11

Rosebrock, A. (2016, November 7). Intersection over Union (IoU) for object detection. PyImageSearch. https://pyimagesearch.com/2016/11/07/intersection-over-union-iou-for-object-detection/

Vovkos. (n.d.). OpenCV showcase. https://vovkos.github.io/doxyrest-showcase/opencv/sphinx_rtd_theme/index.html

ChatGPT. (2024). Prompt: |how to change x1, x2, y1, y2 to x, y, w, h format| Share link. https://chatgpt.com/share/2b0e0c97-7663-4b40-859d-1dafcae32962
